0.01385 1등
0.01946 10등

In [1]:
import pandas as pd
import gc
import numpy as np

In [2]:
def reduce_mem_usage(df):
    # iterate through all the columns of a dataframe and modify the data type
    #   to reduce memory usage.        
    
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [3]:
def featureModify(isTrain):
    if isTrain:
        all_data = pd.read_csv('../train_V2.csv') 
        all_data = all_data[all_data['maxPlace'] > 1]
        all_data = reduce_mem_usage(all_data)
        all_data = all_data[all_data['winPlacePerc'].notnull()]
    else:
        all_data = pd.read_csv('../test_V2.csv')


    all_data['matchType'] = all_data['matchType'].map({
    'crashfpp':1,
    'crashtpp':2,
    'duo':3,
    'duo-fpp':4,
    'flarefpp':5,
    'flaretpp':6,
    'normal-duo':7,
    'normal-duo-fpp':8,
    'normal-solo':9,
    'normal-solo-fpp':10,
    'normal-squad':11,
    'normal-squad-fpp':12,
    'solo':13,
    'solo-fpp':14,
    'squad':15,
    'squad-fpp':16
    })
    all_data = reduce_mem_usage(all_data)

    print("Match size")
    matchSizeData = all_data.groupby(['matchId']).size().reset_index(name='matchSize')
    all_data = pd.merge(all_data, matchSizeData, how='left', on=['matchId'])
    del matchSizeData
    gc.collect()
    
    
    all_data.loc[(all_data['rankPoints']==-1), 'rankPoints'] = 0
    all_data['_killPoints_rankpoints'] = all_data['rankPoints']+all_data['killPoints']


    all_data["_Kill_headshot_Ratio"] = all_data["kills"]/all_data["headshotKills"]
    all_data['_killStreak_Kill_ratio'] = all_data['killStreaks']/all_data['kills']
    all_data['_totalDistance'] = 0.25*all_data['rideDistance'] + all_data["walkDistance"] + all_data["swimDistance"]
    all_data['_killPlace_MaxPlace_Ratio'] = all_data['killPlace'] / all_data['maxPlace']
    all_data['_totalDistance_weaponsAcq_Ratio'] = all_data['_totalDistance'] / all_data['weaponsAcquired']
    all_data['_walkDistance_heals_Ratio'] = all_data['walkDistance'] / all_data['heals']
    all_data['_walkDistance_kills_Ratio'] = all_data['walkDistance'] / all_data['kills']
    all_data['_kills_walkDistance_Ratio'] = all_data['kills'] / all_data['walkDistance']
    all_data['_totalDistancePerDuration'] =  all_data["_totalDistance"]/all_data["matchDuration"]
    all_data['_killPlace_kills_Ratio'] = all_data['killPlace']/all_data['kills']
    all_data['_walkDistancePerDuration'] =  all_data["walkDistance"]/all_data["matchDuration"]
    all_data['walkDistancePerc'] = all_data.groupby('matchId')['walkDistance'].rank(pct=True).values
    all_data['killPerc'] = all_data.groupby('matchId')['kills'].rank(pct=True).values
    all_data['killPlacePerc'] = all_data.groupby('matchId')['killPlace'].rank(pct=True).values
    all_data['weaponsAcquired'] = all_data.groupby('matchId')['weaponsAcquired'].rank(pct=True).values
    all_data['_walkDistance_kills_Ratio2'] = all_data['walkDistancePerc'] / all_data['killPerc']
    all_data['_kill_kills_Ratio2'] = all_data['killPerc']/all_data['walkDistancePerc']
    all_data['_killPlace_walkDistance_Ratio2'] = all_data['walkDistancePerc']/all_data['killPlacePerc']
    all_data['_killPlace_kills_Ratio2'] = all_data['killPlacePerc']/all_data['killPerc']
    all_data['_totalDistance'] = all_data.groupby('matchId')['_totalDistance'].rank(pct=True).values
    all_data['_walkDistance_kills_Ratio3'] = all_data['walkDistancePerc'] / all_data['kills']
    all_data['_walkDistance_kills_Ratio4'] = all_data['kills'] / all_data['walkDistancePerc']
    all_data['_walkDistance_kills_Ratio5'] = all_data['killPerc'] / all_data['walkDistance']
    all_data['_walkDistance_kills_Ratio6'] = all_data['walkDistance'] / all_data['killPerc']

    all_data[all_data == np.Inf] = np.NaN
    all_data[all_data == np.NINF] = np.NaN
    all_data.fillna(0, inplace=True)
    
    features = list(all_data.columns)
    features.remove("Id")
    features.remove("matchId")
    features.remove("groupId")
    features.remove("matchSize")
    features.remove("matchType")
    if isTrain:
        features.remove("winPlacePerc")

    
    print("Mean Data")
    meanData = all_data.groupby(['matchId','groupId'])[features].agg('mean')
    meanData = reduce_mem_usage(meanData)
    meanData = meanData.replace([np.inf, np.NINF,np.nan], 0)
    meanDataRank = meanData.groupby('matchId')[features].rank(pct=True).reset_index()
    meanDataRank = reduce_mem_usage(meanDataRank)
    all_data = pd.merge(all_data, meanData.reset_index(), suffixes=["", "_mean"], how='left', on=['matchId', 'groupId'])
    del meanData
    gc.collect()
    all_data = all_data.drop(["vehicleDestroys_mean","rideDistance_mean","roadKills_mean","rankPoints_mean"], axis=1)
    all_data = pd.merge(all_data, meanDataRank, suffixes=["", "_meanRank"], how='left', on=['matchId', 'groupId'])
    del meanDataRank
    gc.collect()
    all_data = all_data.drop(["numGroups_meanRank","rankPoints_meanRank"], axis=1)
    
    all_data = all_data.join(reduce_mem_usage(all_data.groupby('matchId')[features].rank(ascending=False).add_suffix('_rankPlace').astype(int)))

    
    print("Std Data")
    stdData = all_data.groupby(['matchId','groupId'])[features].agg('std').replace([np.inf, np.NINF,np.nan], 0)
    stdDataRank = reduce_mem_usage(stdData.groupby('matchId')[features].rank(pct=True)).reset_index()
    del stdData
    gc.collect()
    all_data = pd.merge(all_data, stdDataRank, suffixes=["", "_stdRank"], how='left', on=['matchId', 'groupId'])
    del stdDataRank
    gc.collect()
    
    print("Max Data")
    maxData = all_data.groupby(['matchId','groupId'])[features].agg('max')
    maxData = reduce_mem_usage(maxData)
    maxDataRank = maxData.groupby('matchId')[features].rank(pct=True).reset_index()
    maxDataRank = reduce_mem_usage(maxDataRank)
    all_data = pd.merge(all_data, maxData.reset_index(), suffixes=["", "_max"], how='left', on=['matchId', 'groupId'])
    del maxData
    gc.collect()
    all_data = all_data.drop(["assists_max","killPoints_max","headshotKills_max","numGroups_max","revives_max","teamKills_max","roadKills_max","vehicleDestroys_max"], axis=1)
    all_data = pd.merge(all_data, maxDataRank, suffixes=["", "_maxRank"], how='left', on=['matchId', 'groupId'])
    del maxDataRank
    gc.collect()
    all_data = all_data.drop(["roadKills_maxRank","matchDuration_maxRank","maxPlace_maxRank","numGroups_maxRank"], axis=1)


    print("Min Data")
    minData = all_data.groupby(['matchId','groupId'])[features].agg('min')
    minData = reduce_mem_usage(minData)
    minDataRank = minData.groupby('matchId')[features].rank(pct=True).reset_index()
    minDataRank = reduce_mem_usage(minDataRank)
    all_data = pd.merge(all_data, minData.reset_index(), suffixes=["", "_min"], how='left', on=['matchId', 'groupId'])
    del minData
    gc.collect()
    all_data = all_data.drop(["heals_min","killStreaks_min","killPoints_min","maxPlace_min","revives_min","headshotKills_min","weaponsAcquired_min","_walkDistance_kills_Ratio_min","rankPoints_min","matchDuration_min","teamKills_min","numGroups_min","assists_min","roadKills_min","vehicleDestroys_min"], axis=1)
    all_data = pd.merge(all_data, minDataRank, suffixes=["", "_minRank"], how='left', on=['matchId', 'groupId'])
    del minDataRank
    gc.collect()
    all_data = all_data.drop(["killPoints_minRank","matchDuration_minRank","maxPlace_minRank","numGroups_minRank"], axis=1)

    
    print("group Size")
    groupSize = all_data.groupby(['matchId','groupId']).size().reset_index(name='group_size')
    groupSize = reduce_mem_usage(groupSize)
    all_data = pd.merge(all_data, groupSize, how='left', on=['matchId', 'groupId'])
    del groupSize
    gc.collect()

    
    print("Match Mean")
    matchMeanFeatures = features
    matchMeanFeatures = [ v for v in matchMeanFeatures if v not in ["killPlacePerc","matchDuration","maxPlace","numGroups"] ]
    matchMeanData= reduce_mem_usage(all_data.groupby(['matchId'])[matchMeanFeatures].transform('mean')).replace([np.inf, np.NINF,np.nan], 0)
    all_data = pd.concat([all_data,matchMeanData.add_suffix('_matchMean')],axis=1)
    del matchMeanData,matchMeanFeatures
    gc.collect()

    print("matchMax")
    matchMaxFeatures = ["walkDistance","kills","_walkDistance_kills_Ratio","_kill_kills_Ratio2"]
    all_data = pd.merge(all_data, reduce_mem_usage(all_data.groupby(['matchId'])[matchMaxFeatures].agg('max')).reset_index(), suffixes=["", "_matchMax"], how='left', on=['matchId'])

    print("match STD")
    matchMaxFeatures = ["kills","_walkDistance_kills_Ratio2","_walkDistance_kills_Ratio","killPerc","_kills_walkDistance_Ratio"]
    all_data = pd.merge(all_data, reduce_mem_usage(all_data.groupby(['matchId'])[matchMaxFeatures].agg('std')).reset_index().replace([np.inf, np.NINF,np.nan], 0), suffixes=["", "_matchSTD"], how='left', on=['matchId'])


    all_data = all_data.drop(["Id","groupId"], axis=1)
    all_data = all_data.drop(["DBNOs","assists","headshotKills","heals","killPoints","_killStreak_Kill_ratio","killStreaks","longestKill","revives","roadKills","teamKills","vehicleDestroys","_walkDistance_kills_Ratio","weaponsAcquired"], axis=1)
    all_data = all_data.drop(["_walkDistance_heals_Ratio","_totalDistancePerDuration","_killPlace_kills_Ratio","_totalDistance_weaponsAcq_Ratio","_killPlace_MaxPlace_Ratio","_walkDistancePerDuration","rankPoints","rideDistance","boosts","winPoints","swimDistance","_kills_walkDistance_Ratio"], axis=1)
    all_data = all_data.drop(["_Kill_headshot_Ratio","maxPlace","_totalDistance","numGroups","walkDistance","killPlace"], axis=1)
    all_data = reduce_mem_usage(all_data)
    gc.collect()
    
    print("done")
    features_label = all_data.columns
    features_label = features_label.drop('matchId')
    if isTrain:
        features_label = features_label.drop('winPlacePerc')

    gc.collect()
    return all_data,features_label

In [4]:
X_train,features_label = featureModify(True) 

print("Split time")
def split_train_val(data, fraction):
    matchIds = data['matchId'].unique().reshape([-1])
    train_size = int(len(matchIds)*fraction)
    
    random_idx = np.random.RandomState(seed=2).permutation(len(matchIds))
    train_matchIds = matchIds[random_idx[:train_size]]
    val_matchIds = matchIds[random_idx[train_size:]]
    
    data_train = data.loc[data['matchId'].isin(train_matchIds)]
    data_val = data.loc[data['matchId'].isin(val_matchIds)]
    return data_train, data_val

Memory usage of dataframe is 1017.83 MB
Memory usage after optimization is: 322.31 MB
Decreased by 68.3%
Memory usage of dataframe is 322.31 MB
Memory usage after optimization is: 292.63 MB
Decreased by 9.2%
Match size
Mean Data
Memory usage of dataframe is 642.07 MB
Memory usage after optimization is: 216.85 MB
Decreased by 66.2%
Memory usage of dataframe is 757.68 MB
Memory usage after optimization is: 212.61 MB
Decreased by 71.9%
Memory usage of dataframe is 831.23 MB
Memory usage after optimization is: 233.25 MB
Decreased by 71.9%
Std Data
Memory usage of dataframe is 758.04 MB
Memory usage after optimization is: 212.98 MB
Decreased by 71.9%
Max Data
Memory usage of dataframe is 394.67 MB
Memory usage after optimization is: 189.79 MB
Decreased by 51.9%
Memory usage of dataframe is 757.68 MB
Memory usage after optimization is: 212.61 MB
Decreased by 71.9%
Min Data
Memory usage of dataframe is 394.67 MB
Memory usage after optimization is: 189.79 MB
Decreased by 51.9%
Memory usage of 

In [5]:
# Split the Data by matchId.
X_train, X_train_test = split_train_val(X_train, 0.91)
print("Y time")
y = X_train['winPlacePerc']
y_test = X_train_test['winPlacePerc']
print("X_train time")
X_train = X_train.drop(columns=['matchId', 'winPlacePerc'])
print("X test train time")
X_train_test = X_train_test.drop(columns='matchId')
print("X test train winPlace remove")
X_train_test = X_train_test.drop(columns='winPlacePerc')

print("X test np time")
X_train_test = np.array(X_train_test)
print("y test np time")
y_test = np.array(y_test)

Y time
X_train time
X test train time
X test train winPlace remove
X test np time
y test np time


In [6]:
#Split the Data again and then join it. I am doing this because If I turn the Pandas DataFrame into Numpy Array with 
# all rows at once, Kernel will be killed for exceeding 16GB Memory. 
from sklearn.model_selection import train_test_split
X_train, X_train2, y, y2 = train_test_split(X_train, y, test_size=0.1, shuffle=False)
print("X_train np time")
X_train = np.array(X_train)
print("y np time")
y = np.array(y)

print("X_train2 np time")
X_train2 = np.array(X_train2)
print("y2 np time")
y2 = np.array(y2)

y = np.concatenate((y, y2), axis=0)
del y2
gc.collect()
X_train = np.concatenate((X_train, X_train2), axis=0)
del X_train2
gc.collect()

X_train np time
y np time
X_train2 np time
y2 np time


0

In [7]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error

In [8]:
lr_reg = LinearRegression()
lr_reg.fit(X_train, y)

ridge_reg = Ridge()
ridge_reg.fit(X_train, y)

lasso_reg = Lasso()
lasso_reg.fit(X_train, y)

Lasso()

In [9]:
def get_rmse(model):
    pred = model.predict(X_train_test)
    mse = mean_squared_error(y_test , pred)
    rmse = np.sqrt(mse)
    print('{0} 로그 변환된 RMSE: {1}'.format(model.__class__.__name__,np.round(rmse, 3)))
    return rmse

def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses

In [10]:
models = [lr_reg, ridge_reg, lasso_reg]
get_rmses(models)

LinearRegression 로그 변환된 RMSE: 0.05000000074505806
Ridge 로그 변환된 RMSE: 0.04800000041723251
Lasso 로그 변환된 RMSE: 0.12999999523162842


[0.04968881, 0.048239052, 0.12987678]

# Pycaret

In [11]:
from pycaret.regression import *

In [12]:
setup_clf = setup(data = X_train, target= 'winPlacePerc', fold_shuffle=True)

TypeError: data passed must be of type pandas.DataFrame

In [ ]:
top10 = compare_models(sort='MAE', n_select=10, fold=5) 

IntProgress(value=0, description='Processing: ', max=104)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:15:47
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.0974,0.0177,0.1332,0.8122,0.0895,0.4046,2.4920
ridge,Ridge Regression,0.0974,0.0177,0.1332,0.8122,0.0895,0.4046,0.7060
lar,Least Angle Regression,0.0974,0.0177,0.1332,0.8122,0.0895,0.4048,0.7920
br,Bayesian Ridge,0.0974,0.0177,0.1332,0.8122,0.0895,0.4046,3.0700
en,Elastic Net,0.1191,0.0234,0.1531,0.7520,0.1046,0.5704,1.1300
omp,Orthogonal Matching Pursuit,0.1192,0.0240,0.1548,0.7465,0.1056,0.5382,0.8060
lasso,Lasso Regression,0.1263,0.0257,0.1604,0.7277,0.1101,0.6241,1.2760
par,Passive Aggressive Regressor,0.1286,0.0320,0.1781,0.6617,0.1128,0.4207,6.0280
huber,Huber Regressor,0.2111,0.0916,0.2971,0.0300,0.1713,0.8135,48.1660
llar,Lasso Least Angle Regression,0.2679,0.0945,0.3074,-0.0000,0.2140,1.5351,0.8220


In [ ]:
tuned_top5 = [tune_model(i, fold=5) for i in top10[:5]]

In [ ]:
blender_top5 = blend_models(estimator_list=tuned_top5) 

In [ ]:
final_model = finalize_model(blender_top5)
prediction = predict_model(final_model, data=test)

In [ ]:
from pycaret.utils import check_metric
check_metric(prediction['Target'], prediction['Label'], metric = 'RMSE') 